Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

## Abstractive Summarization on CNN/DM Dataset using Transformers


### Summary

This notebook demonstrates how to fine tune Transformers for extractive text summarization. Utility functions and classes in the NLP Best Practices repo are used to facilitate data preprocessing, model training, model scoring, result postprocessing, and model evaluation.




### Before You Start

The running time shown in this notebook is on a Standard_NC24s_v3 Azure Ubuntu Virtual Machine with 4 NVIDIA Tesla V100 GPUs. 
> **Tip**: If you want to run through the notebook quickly, you can set the **`QUICK_RUN`** flag in the cell below to **`True`** to run the notebook on a small subset of the data and a smaller number of epochs. 

Using only 1 NVIDIA Tesla V100 GPUs, 16GB GPU memory configuration,
- for data preprocessing, it takes around 1 minutes to preprocess the data for quick run. Otherwise it takes ~20 minutes to finish the data preprocessing. This time estimation assumes that the chosen transformer model is "distilbert-base-uncased" and the sentence selection method is "greedy", which is the default. The preprocessing time can be significantly longer if the sentence selection method is "combination", which can achieve better model performance.

- for model fine tuning, it takes around 2 minutes for quick run. Otherwise, it takes around ~3 hours to finish. This estimation assumes the chosen encoder method is "transformer". The model fine tuning time can be shorter if other encoder method is chosen, which may result in worse model performance. 

### Additional Notes

* **ROUGE Evalation**: To run rouge evaluation, please refer to the section of compute_rouge_perl in [summarization_evaluation.ipynb](./summarization_evaluation.ipynb) for setup.

* **Distributed Training**:
Please note that the jupyter notebook only allows to use pytorch [DataParallel](https://pytorch.org/docs/master/nn.html#dataparallel). Faster speed and larger batch size can be achieved with pytorch [DistributedDataParallel](https://pytorch.org/docs/master/notes/ddp.html)(DDP). Script [extractive_summarization_cnndm_distributed_train.py](./extractive_summarization_cnndm_distributed_train.py) shows an example of how to use DDP.



In [1]:
%load_ext autoreload

%autoreload 2
## Set QUICK_RUN = True to run the notebook on a small subset of data and a smaller number of epochs.
QUICK_RUN = True


### Configuration


In [4]:
import os
import shutil
import sys
from tempfile import TemporaryDirectory
import torch

nlp_path = os.path.abspath("../../")
if nlp_path not in sys.path:
    sys.path.insert(0, nlp_path)

from utils_nlp.dataset.cnndm import CNNDMBertSumProcessedData, CNNDMSummarizationDataset
from utils_nlp.eval import compute_rouge_python, compute_rouge_perl
from utils_nlp.models.transformers.abstractive_summarization_bartt5 import AbstractiveSummarizer, SummarizationProcessor

from utils_nlp.models.transformers.datasets import SummarizationDataset
import nltk
from nltk import tokenize

import pandas as pd
import scrapbook as sb
import pprint


### Configuration: choose the transformer model to be used

Several pretrained models have been made available by [Hugging Face](https://github.com/huggingface/transformers). For extractive summarization, the following pretrained models are supported. 

In [3]:
#pd.DataFrame({"model_name": ExtractiveSummarizer.list_supported_models()})

In [8]:
# Transformer model being used
MODEL_NAME = "t5-large"

In [11]:
# notebook parameters
# the cache data path during find tuning
CACHE_DIR = TemporaryDirectory().name
        
processor = SummarizationProcessor(MODEL_NAME,cache_dir=CACHE_DIR ) #tokenizer, config.prefix)

### Data Preprocessing

The dataset we used for this notebook is CNN/DM dataset which contains the documents and accompanying questions from the news articles of CNN and Daily mail. The highlights in each article are used as summary. The dataset consits of ~289K training examples, ~11K valiation examples and ~11K test examples.   The code in following cell will download the CNN/DM dataset listed at https://github.com/harvardnlp/sent-summary/.


In [12]:
# the data path used to save the downloaded data file
DATA_PATH = TemporaryDirectory().name
# The number of lines at the head of data file used for preprocessing. -1 means all the lines.
TOP_N = 1000
if not QUICK_RUN:
    TOP_N = -1

In [13]:
train_dataset, test_dataset = CNNDMSummarizationDataset(top_n=TOP_N, local_cache_path=DATA_PATH, raw=True)

100%|██████████| 489k/489k [00:08<00:00, 56.3kKB/s] 


Preprocess the data.

In [14]:

abs_sum_train = processor.preprocess(train_dataset)
abs_sum_test = processor.preprocess(test_dataset)


In [9]:
"""
# save and load preprocessed data
save_path = os.path.join(DATA_PATH, "processed")
torch.save(ext_sum_train, os.path.join(save_path, "train_full.pt"))
torch.save(ext_sum_test, os.path.join(save_path, "test_full.pt"))

"""
# ext_sum_train = torch.load(os.path.join(save_path, "train_full.pt"))
# ext_sum_test = torch.load(os.path.join(save_path, "test_full.pt"))

'\n# save and load preprocessed data\nsave_path = os.path.join(DATA_PATH, "processed")\ntorch.save(ext_sum_train, os.path.join(save_path, "train_full.pt"))\ntorch.save(ext_sum_test, os.path.join(save_path, "test_full.pt"))\n\n'

In [10]:
print(len(abs_sum_train))
print(len(abs_sum_test))

1000
1000


#### Inspect Data

In [11]:
abs_sum_train[0].keys()

dict_keys(['src', 'src_txt', 'tgt', 'tgt_txt', 'source_ids', 'source_mask', 'target_ids'])

In [12]:
abs_sum_train[0]

{'src': "editor 's note : in our behind the scenes series , cnn correspondents share their experiences in covering news and analyze the stories behind the events . here , soledad o'brien takes users inside a jail where many of the inmates are mentally ill . an inmate housed on the `` forgotten floor , '' where many mentally ill inmates are housed in miami before trial . miami , florida -lrb- cnn -rrb- -- the ninth floor of the miami-dade pretrial detention facility is dubbed the `` forgotten floor . '' here , inmates with the most severe mental illnesses are incarcerated until they 're ready to appear in court . most often , they face drug charges or charges of assaulting an officer -- charges that judge steven leifman says are usually `` avoidable felonies . '' he says the arrests often result from confrontations with police . mentally ill people often wo n't do what they 're told when police arrive on the scene -- confrontation seems to exacerbate their illness and they become more p

### Model training
To start model training, we need to create a instance of ExtractiveSummarizer.
#### Choose the transformer model.
Currently ExtractiveSummarizer support two models:
- distilbert-base-uncase, 
- bert-base-uncase

Potentionally, roberta-based model and xlnet can be supported but needs to be tested.
#### Choose the encoder algorithm.
There are four options:
- baseline: it used a smaller transformer model to replace the bert model and with transformer summarization layer
- classifier: it uses pretrained BERT and fine-tune BERT with **simple logistic classification** summarization layer
- transformer: it uses pretrained BERT and fine-tune BERT with **transformer** summarization layer
- RNN: it uses pretrained BERT and fine-tune BERT with **LSTM** summarization layer

In [32]:
BATCH_SIZE = 5 # batch size, unit is the number of samples
MAX_POS_LENGTH = 512

    


# GPU used for training
NUM_GPUS = torch.cuda.device_count()


# Learning rate
LEARNING_RATE=2e-3

# How often the statistics reports show up in training, unit is step.
REPORT_EVERY=100

# total number of steps for training
MAX_STEPS=1e2
# number of steps for warm up
WARMUP_STEPS=5e2
    
if not QUICK_RUN:
    MAX_STEPS=5e4
    WARMUP_STEPS=5e3
 

In [15]:
summarizer = AbstractiveSummarizer(processor, MODEL_NAME)

In [15]:
"""

summarizer.fit(
            ext_sum_train,
            num_gpus=NUM_GPUS,
            batch_size=BATCH_SIZE,
            gradient_accumulation_steps=2,
            max_steps=MAX_STEPS,
            learning_rate=LEARNING_RATE,
            warmup_steps=WARMUP_STEPS,
            verbose=True,
            report_every=REPORT_EVERY,
            clip_grad_norm=False,
            use_preprocessed_data=USE_PREPROCSSED_DATA
        )

"""


'\n\nsummarizer.fit(\n            ext_sum_train,\n            num_gpus=NUM_GPUS,\n            batch_size=BATCH_SIZE,\n            gradient_accumulation_steps=2,\n            max_steps=MAX_STEPS,\n            learning_rate=LEARNING_RATE,\n            warmup_steps=WARMUP_STEPS,\n            verbose=True,\n            report_every=REPORT_EVERY,\n            clip_grad_norm=False,\n            use_preprocessed_data=USE_PREPROCSSED_DATA\n        )\n\n'

In [16]:
"""
summarizer.save_model(
    os.path.join(
        CACHE_DIR,
        "extsum_modelname_{0}_usepreprocess{1}_steps_{2}.pt".format(
            MODEL_NAME, USE_PREPROCSSED_DATA, MAX_STEPS
        ),
    )
)
"""


'\nsummarizer.save_model(\n    os.path.join(\n        CACHE_DIR,\n        "extsum_modelname_{0}_usepreprocess{1}_steps_{2}.pt".format(\n            MODEL_NAME, USE_PREPROCSSED_DATA, MAX_STEPS\n        ),\n    )\n)\n'

In [17]:
# for loading a previous saved model
"""
import torch
model_path = os.path.join(
        CACHE_DIR,
        "extsum_modelname_{0}_usepreprocess{1}_steps_{2}.pt".format(
            MODEL_NAME, USE_PREPROCSSED_DATA, MAX_STEPS
        ))
summarizer = ExtractiveSummarizer(processor, MODEL_NAME, ENCODER, MAX_POS_LENGTH, CACHE_DIR)
summarizer.model.load_state_dict(torch.load(model_path, map_location="cpu"))
"""

'\nimport torch\nmodel_path = os.path.join(\n        CACHE_DIR,\n        "extsum_modelname_{0}_usepreprocess{1}_steps_{2}.pt".format(\n            MODEL_NAME, USE_PREPROCSSED_DATA, MAX_STEPS\n        ))\nsummarizer = ExtractiveSummarizer(processor, MODEL_NAME, ENCODER, MAX_POS_LENGTH, CACHE_DIR)\nsummarizer.model.load_state_dict(torch.load(model_path, map_location="cpu"))\n'

### Model Evaluation

[ROUGE](https://en.wikipedia.org/wiki/ROUGE_(metric)), or Recall-Oriented Understudy for Gisting Evaluation has been commonly used for evaluating text summarization.

In [18]:
abs_sum_test[0].keys()

dict_keys(['src', 'src_txt', 'tgt', 'tgt_txt', 'source_ids', 'source_mask', 'target_ids'])

In [19]:
source = []
target = []
for i in abs_sum_test:
    source.append(i["src_txt"]) 
    target.append(i['tgt']) 

In [20]:
target[0]

"<t> marseille prosecutor says `` so far no videos were used in the crash investigation '' despite media reports . </t> <t> journalists at bild and paris match are `` very confident '' the video clip is real , an editor says . </t> <t> andreas lubitz had informed his lufthansa training school of an episode of severe depression , airline says . </t>\n"

In [19]:
%%time
prediction = summarizer.predict(abs_sum_test[0:20], num_gpus=2, batch_size=10)

Generating summary:   0%|          | 0/2 [00:00<?, ?it/s]

dataset length is 20
{'source_ids': tensor([[21603,    10,  8113,  ...,  2493,     3,    17],
        [21603,    10,     3,  ...,     0,     0,     0],
        [21603,    10,     3,  ...,    52,   115,    18],
        ...,
        [21603,    10,     3,  ...,     0,     0,     0],
        [21603,    10,     3,  ...,   564,     3,     5],
        [21603,    10,     3,  ...,     0,     0,     0]], device='cuda:0'), 'source_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0]], device='cuda:0')}


Generating summary:  50%|█████     | 1/2 [00:19<00:19, 19.05s/it]

{'source_ids': tensor([[21603,    10,     3,  ...,     0,     0,     0],
        [21603,    10,     3,  ...,     0,     0,     0],
        [21603,    10, 10381,  ...,  1112,   300,   270],
        ...,
        [21603,    10,     3,  ...,     0,     0,     0],
        [21603,    10,  3701,  ...,  3579,  2634,     3],
        [21603,    10,     3,  ...,     0,     0,     0]], device='cuda:0'), 'source_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0]], device='cuda:0')}


Generating summary: 100%|██████████| 2/2 [00:38<00:00, 19.05s/it]


CPU times: user 31.6 s, sys: 9.21 s, total: 40.8 s
Wall time: 39.4 s


In [22]:
len(prediction)

20

In [20]:
prediction

["marseille , france - cnn - the french prosecutor leading an investigation into the crash of germanwings flight 9525 insisted wednesday that he was not aware of any video footage from on board the plane . marseille prosecutor brice robin told cnn that  so far no videos were used in the crash investigation . '' robin 's comments follow claims by two magazines , german daily bild and french paris match , of",
 "the formal accession was marked with a ceremony at the hague , in the netherlands , where the court is based . the palestinians signed the icc's founding rome statute in january . as members of the court , palestinians may be subject to counter-charges as well .",
 "amnesty international's annual report catalogs the use of state-sanctioned killing as a punitive measure across the globe . at least 607 people were executed around the world in 2014 . at least 2,466 people globally are confirmed to have been handed the sentence .",
 'amnesty international releases annual review of de

In [ ]:
len()

In [18]:
prediction

['marseille , france - cnn - the french prosecutor leading',
 'the formal accession was marked with a ceremony at the hague , in the net',
 "amnesty international's annual report catalogs the use of state-sanction",
 'amnesty international releases annual review of death penalty worldwide . death sentences up 500%',
 'anne frank died of typhus in a nazi concentration camp at the',
 'a duke student has admitted to hanging a noose made of rope from',
 'the rev. robert h. schuller , californ',
 'the dog was hit by a car, apparently killed with a hammer and ',
 'mohammad javad zarif is the iranian foreign minister ',
 'for the first time in eight years , a tv legend returned to doing what',
 "-lrb- he 's a blue chip college basketball recruit ",
 'michele bachmann compared president obama to the co-pilot of',
 'california is a breadbasket to the nation . cali',
 "walmart's staunch criticism of a religious freedom law in its home state of ark",
 'five americans who were monitored for three 

In [43]:
prediction

['bild and paris match claim to have found a video of the crash site .',
 'palestinian authority officially became the 123rd member of the international criminal court on we',
 "amnesty international's annual report catalogs the use of state-sanction",
 '. in egypt and nigeria , courts imposed',
 'anne frank died of typhus in a nazi concentration camp at the',
 'student admitted to hanging the noose from a tree near a student union ',
 ". schuller 's legacy is a televangelist ",
 '. a dog in california , found seemingly dead after',
 'zarif is the iranian foreign minister . he has been ',
 ", he didn't seem to miss a beat . bob bark",
 '. . . . . . . . .',
 'michele bachmann compared president obama to the co-pilot of',
 '. . . . . . . . .',
 "''  the republican party will have to better stand for '' ideas",
 'of an infected person . the last of 17 patients who were being monitored are',
 'found dead in his home . he was found on his side near a bathroom',
 '. . . . . . . . ',
 ". ''we

In [55]:
prediction

['marseille , france - cnn - the french prosecutor leading',
 'the formal accession was marked with a ceremony at the hague , in the net',
 "amnesty international's annual report catalogs the use of state-sanction",
 'amnesty international releases annual review of death penalty worldwide . death sentences up 500%',
 'anne frank died of typhus in a nazi concentration camp at the',
 'a duke student has admitted to hanging a noose made of rope from',
 'the rev. robert h. schuller , californ',
 'the dog was hit by a car, apparently killed with a hammer and ',
 'mohammad javad zarif is the iranian foreign minister ',
 'for the first time in eight years , a tv legend returned to doing what',
 "-lrb- he 's a blue chip college basketball recruit ",
 'michele bachmann compared president obama to the co-pilot of',
 'california is a breadbasket to the nation . cali',
 "walmart's staunch criticism of a religious freedom law in its home state of ark",
 'five americans who were monitored for three 

In [24]:
rouge_scores = compute_rouge_python(cand=prediction, ref=target)
pprint.pprint(rouge_scores)

Number of candidates: 20
Number of references: 1000


AssertionError: 

In [ ]:
target[0]

In [ ]:
prediction[0]

In [ ]:
source[0]

In [ ]:
# for testing
sb.glue("rouge_2_f_score", rouge_scores['rouge-2']['f'])

## Prediction on a single input sample

In [ ]:
source = """
But under the new rule, set to be announced in the next 48 hours, Border Patrol agents would immediately return anyone to Mexico — without any detainment and without any due process — who attempts to cross the southwestern border between the legal ports of entry. The person would not be held for any length of time in an American facility.

Although they advised that details could change before the announcement, administration officials said the measure was needed to avert what they fear could be a systemwide outbreak of the coronavirus inside detention facilities along the border. Such an outbreak could spread quickly through the immigrant population and could infect large numbers of Border Patrol agents, leaving the southwestern border defenses weakened, the officials argued.
The Trump administration plans to immediately turn back all asylum seekers and other foreigners attempting to enter the United States from Mexico illegally, saying the nation cannot risk allowing the coronavirus to spread through detention facilities and Border Patrol agents, four administration officials said.
The administration officials said the ports of entry would remain open to American citizens, green-card holders and foreigners with proper documentation. Some foreigners would be blocked, including Europeans currently subject to earlier travel restrictions imposed by the administration. The points of entry will also be open to commercial traffic."""

In [ ]:
test_dataset = SummarizationDataset(
    None,
    source=[source],
    source_preprocessing=[tokenize.sent_tokenize],
    word_tokenize=nltk.word_tokenize,
)
processor = ExtSumProcessor(model_name=MODEL_NAME,  cache_dir=CACHE_DIR)
preprocessed_dataset = processor.preprocess(test_dataset)

In [ ]:
preprocessed_dataset[0].keys()

In [ ]:
prediction = summarizer.predict(preprocessed_dataset, num_gpus=0, batch_size=1, sentence_separator="\n")

In [ ]:
prediction

## Clean up temporary folders

In [ ]:
if os.path.exists(DATA_PATH):
    shutil.rmtree(DATA_PATH, ignore_errors=True)
if os.path.exists(CACHE_DIR):
    shutil.rmtree(CACHE_DIR, ignore_errors=True)
if USE_PREPROCSSED_DATA:
    if os.path.exists(PROCESSED_DATA_PATH):
        shutil.rmtree(PROCESSED_DATA_PATH, ignore_errors=True)